# 분류 - classification
    카테고리를 갖는 경우
    KNN, SVM, Decision Tree, Logistic Regression

### KNN K-Nearest-Neighbors

# 1. 데이터 준비(확보)

In [135]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [136]:
titanic = sns.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [137]:
titanic.rename(columns={'sex':'gender'}, inplace=True)
titanic.head()

,survived,pclass,gender,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


# 2. 데이터 탐색 - info(), describe()

In [138]:
# 결측치 확인 - 갯수, 형태는 숫자인데 타입이 object면 의심해 봐라
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   gender       891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


### 결측치가 많은 column은 통으로 제거 - deck 제거
### 의미가 비슷한 column은 하나만 사용 - embark_town 제거

In [139]:
titanic.drop(['deck','embark_town'], axis=1, inplace=True)

In [140]:
titanic.isna().sum()

survived        0
pclass          0
gender          0
age           177
sibsp           0
parch           0
fare            0
embarked        2
class           0
who             0
adult_male      0
alive           0
alone           0
dtype: int64

In [141]:
freq = titanic['embarked'].value_counts().idxmax()

In [142]:
titanic['embarked'].fillna(freq, inplace=True)

In [143]:
titanic.isnull().sum()

survived        0
pclass          0
gender          0
age           177
sibsp           0
parch           0
fare            0
embarked        0
class           0
who             0
adult_male      0
alive           0
alone           0
dtype: int64

In [144]:
# age 처리여부
# 1. drop
# titanic.dropna(subset=['age']).isnull().sum()
# 2. 치환 - 선실등급(pclass)별 gender별 평균으로 대체
grouped = titanic.groupby(['pclass', 'gender'])

In [216]:
grouped['age'].transform('mean')

0      26.507589
1      34.611765
2      21.750000
3      34.611765
4      26.507589
         ...    
886    30.740707
887    34.611765
888    21.750000
889    41.281386
890    26.507589
Name: age, Length: 891, dtype: float64

In [145]:
grouped_df = grouped['age'].mean()
grouped_df.index

MultiIndex([(1, 'female'),
            (1,   'male'),
            (2, 'female'),
            (2,   'male'),
            (3, 'female'),
            (3,   'male')],
           names=['pclass', 'gender'])

In [146]:
df = titanic[titanic.age.isnull()]

In [147]:
idx1,idx2,idx3,idx4,idx5,idx6 = [],[],[],[],[],[]
for i in range(len(df)):
    if (df.iloc[i,1] == 1) & (df.iloc[i,2] == 'female') :        
        idx1.append(df.index[i])
    elif (df.iloc[i,1] == 1) & (df.iloc[i,2] == 'male') :
        idx2.append(df.index[i])
    elif (df.iloc[i,1] == 2) & (df.iloc[i,2] == 'female') :        
        idx3.append(df.index[i])
    elif (df.iloc[i,1] == 2) & (df.iloc[i,2] == 'male') :
        idx4.append(df.index[i])
    elif (df.iloc[i,1] == 3) & (df.iloc[i,2] == 'female') :        
        idx5.append(df.index[i])
    elif (df.iloc[i,1] == 3) & (df.iloc[i,2] == 'male') :
        idx6.append(df.index[i])    

In [166]:
# idx1 ~ idx5
titanic.iloc[idx1,3] = grouped_df.values[0]
titanic.iloc[idx2,3] = grouped_df.values[1]
titanic.iloc[idx3,3] = grouped_df.values[2]
titanic.iloc[idx4,3] = grouped_df.values[3]
titanic.iloc[idx5,3] = grouped_df.values[4]
titanic.iloc[idx6,3] = grouped_df.values[5]

In [167]:
titanic.age.isnull()

0      False
1      False
2      False
3      False
4      False
       ...  
886    False
887    False
888    False
889    False
890    False
Name: age, Length: 891, dtype: bool

# 3. 속성선택

In [168]:
df = titanic.loc[:,:'embarked']

In [169]:
df.head()

,survived,pclass,gender,age,sibsp,parch,fare,embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [170]:
df.drop(['fare'], axis=1, inplace=True)

In [171]:
onehot_gender = pd.get_dummies(df['gender'])
onehot_embarked = pd.get_dummies(df['embarked'],prefix='town')

In [172]:
df = pd.concat([df, onehot_gender, onehot_embarked],axis=1)

In [174]:
df.head()

,survived,pclass,gender,age,sibsp,parch,embarked,female,male,town_C,town_Q,town_S
0,0,3,male,22.0,1,0,S,0,1,0,0,1
1,1,1,female,38.0,1,0,C,1,0,1,0,0
2,1,3,female,26.0,0,0,S,1,0,0,0,1
3,1,1,female,35.0,1,0,S,1,0,0,0,1
4,0,3,male,35.0,0,0,S,0,1,0,0,1


In [175]:
df.drop(columns=['gender', 'embarked'], inplace=True)

In [176]:
df.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


# 훈련용 / 검증용 데이터 분할

In [177]:
df.isnull().sum()

survived    0
pclass      0
age         0
sibsp       0
parch       0
female      0
male        0
town_C      0
town_Q      0
town_S      0
dtype: int64

In [178]:
x = df.loc[:, 'pclass':]
x
y = df['survived']

In [160]:
from sklearn.model_selection import train_test_split

In [191]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

In [192]:
x_train.isnull().sum()

pclass    0
age       0
sibsp     0
parch     0
female    0
male      0
town_C    0
town_Q    0
town_S    0
dtype: int64

# 모델선택

In [182]:
from sklearn.neighbors import KNeighborsClassifier

In [193]:
knn = KNeighborsClassifier(n_neighbors=5)

# 학습

In [194]:
knn.fit(x_train, y_train)

KNeighborsClassifier()

In [195]:
y_hat = knn.predict(x_test)

In [196]:
y_hat[:10]

array([0, 0, 0, 1, 1, 0, 1, 0, 1, 1], dtype=int64)

In [197]:
y_test.values[:10]

array([1, 0, 0, 1, 1, 1, 1, 0, 1, 1], dtype=int64)

# 평가

In [188]:
from sklearn import metrics

In [198]:
knn_matrix = metrics.confusion_matrix(y_test, y_hat)

In [199]:
knn_matrix

array([[144,  13],
       [ 46,  65]], dtype=int64)

# 정확도(precision)

In [200]:
knn_report = metrics.classification_report(y_test, y_hat)
print(knn_report)

              precision    recall  f1-score   support

           0       0.76      0.92      0.83       157
           1       0.83      0.59      0.69       111

    accuracy                           0.78       268
   macro avg       0.80      0.75      0.76       268
weighted avg       0.79      0.78      0.77       268



### 사망자에 대한 예측능력: 0.83
### 생존자에 대한 예측능력: 0.69

# svm: Support Vector Machine
    벡터개념을 가지고 공간을 분리: 벡터개념은 2차원 평면 공간으로 만들어서 분리

In [201]:
from sklearn import svm

In [202]:
svm_model = svm.SVC()

In [203]:
svm_model.fit(x_train, y_train)

SVC()

In [204]:
y_hat = svm_model.predict(x_test)

In [205]:
print(y_test.values[:10])
print(y_hat[:10])

[1 0 0 1 1 1 1 0 1 1]
[0 0 0 1 0 0 0 0 0 0]


In [207]:
svm_report = metrics.classification_report(y_test, y_hat)
print(svm_report)

              precision    recall  f1-score   support

           0       0.60      0.97      0.74       157
           1       0.64      0.06      0.11       111

    accuracy                           0.60       268
   macro avg       0.62      0.52      0.43       268
weighted avg       0.61      0.60      0.48       268



In [208]:
from sklearn.preprocessing import StandardScaler

In [211]:
ss = StandardScaler()
x_train_scaled = ss.fit_transform(x_train)
x_test_scaled = ss.fit_transform(x_test)

In [212]:
svm_model.fit(x_train_scaled, y_train)

SVC()

In [213]:
y_hat = svm_model.predict(x_test_scaled)